# Processando entradas do dataframe de métricas

## Importando os módulos necessários

In [1]:
%pip install openpyxl

from tensorflow import keras
import tensorflow_hub as hub
import pandas as pd

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


2023-06-12 15:54:44.747495: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-12 15:54:44.776961: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-12 15:54:44.777847: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-12 15:54:45.333327: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


## Carregando modelo já criado em "model-making-script"

In [22]:
searchModel = keras.models.load_model(
    '../model-making-scripts/searchModel.h5', custom_objects={'KerasLayer': hub.KerasLayer})

2023-06-12 16:16:58.061930: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype string
	 [[{{node inputs}}]]
2023-06-12 16:16:58.771367: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder' with dtype string and shape [?]
	 [[{{node Placeholder}}]]


## Carregando os dados processados em json

In [23]:
import json

data_file_path = './data/data.json'

def getCoursesData():

    with open(data_file_path) as json_file:
        data = json.load(json_file)
    
    return data

coursesData = getCoursesData()

## Carregando o dataframe de métrica e acessando as entradas.
+ O dataframe de métrica possui uma coluna com uma pesquisa usada para buscar um artigo no espaço vetorial
+ Possui também mais duas colunas, um curso esperado e um indice para comparação
+ A coluna curso retornado e saída, são para a comparação dos dados de entrada e saída, caso o curso retornado seja igual ao esperado, a saída é 1, caso contrário é 0.

In [24]:
df = pd.read_excel('./data/Dataframe para Métrica.xlsx')

entradas = df['entrada (português)']

df

,entrada (português),índice,curso esperado,curso retornado,saida
0,Quais as modalidades de cursos de graduacao de...,0,Art. 1 Os cursos de graduação da Universidade ...,NaN,NaN
1,Quem estabelece as diretrizes curriculares?,1,Os cursos de graduação poderão oferecer habili...,NaN,NaN
2,Como sao organizadas as atividades academicas ...,2,Os cursos de graduação terão suas atividades a...,NaN,NaN
3,O que visam os cursos de graduacao,3,Os cursos de graduação visam à obtenção de qua...,NaN,NaN
4,Quem regulamenta a estrutura curricular de cad...,4,A estrutura curricular de cada curso de gradua...,NaN,NaN
...,...,...,...,...,...
192,Sou do exterior posso transferir pra ufcg,11,Será permitida a transferência para cursos da ...,NaN,NaN
193,Posso transferir pra ufcg,11,Será permitida a transferência para cursos da ...,NaN,NaN
194,Quais modalidades de transferencia,11,Será permitida a transferência para cursos da ...,NaN,NaN
195,O processo seletivo pra entrar na ufcg tem num...,12,A admissão de alunos aos cursos de graduação d...,NaN,NaN


## Processando as entradas e atribuindo os resultados na coluna "curso retornado"
+ O resultado das buscas é feito com base na função de similaridade do cosseno

In [31]:
import numpy as np
from scipy.stats import pearsonr

retornos = []

for entrada in entradas:
    
    entrada_str = str(entrada)

    retorno = searchModel.predict([entrada_str])  # Faça a previsão com base nas entradas
    retorno = np.array(retorno)  # Converte para um array numpy
    similarity_scores = []
    for course_data in coursesData["data"]:
        course_data = np.array(course_data)  # Converte para um array numpy
        similarity_scores.append(pearsonr(retorno.flatten(), course_data.flatten())[0])
    indexes = np.argsort(similarity_scores)[::-1]  # Ordenar em ordem decrescente
    retornos.append(coursesData['num'][indexes[0]])

    
df['curso retornado'] = retornos

df


1/1 [==============================] - 0s 34ms/step


,entrada (português),índice,curso esperado,curso retornado,saida
0,Quais as modalidades de cursos de graduacao de...,0,Art. 1 Os cursos de graduação da Universidade ...,Nas disciplinas cuja avaliação final é realiza...,0
1,Quem estabelece as diretrizes curriculares?,1,Os cursos de graduação poderão oferecer habili...,Será permitido ao aluno provável concluinte no...,0
2,Como sao organizadas as atividades academicas ...,2,Os cursos de graduação terão suas atividades a...,As vagas remanescentes em disciplinas após o e...,0
3,O que visam os cursos de graduacao,3,Os cursos de graduação visam à obtenção de qua...,Nas disciplinas cuja avaliação final é realiza...,0
4,Quem regulamenta a estrutura curricular de cad...,4,A estrutura curricular de cada curso de gradua...,A matrícula em disciplinas far-se-á dentre um ...,0
...,...,...,...,...,...
192,Sou do exterior posso transferir pra ufcg,11,Será permitida a transferência para cursos da ...,"A matrícula em disciplinas será realizada, por...",0
193,Posso transferir pra ufcg,11,Será permitida a transferência para cursos da ...,A matrícula institucional poderá ser concedida...,0
194,Quais modalidades de transferencia,11,Será permitida a transferência para cursos da ...,A abreviação da duração do curso de graduação ...,0
195,O processo seletivo pra entrar na ufcg tem num...,12,A admissão de alunos aos cursos de graduação d...,"A matrícula em disciplinas será realizada, por...",0


In [ ]:
# !pip install unidecode
# from unidecode import unidecode

# ignore_list = ['the', 'o', 'a']

# def parse_name(full_name):
#     name_list = full_name.split() # Separa cada nome
#     new_name_list = []
#     for name in name_list: # Percorre cada nome
#         name = name.strip('.') # Remove pontos
#         name = name.lower() # Converte todas as letras em minúsculas
#         if name in ignore_list: # Remove preposições
#             continue
        # name = unidecode(name.decode('utf8')) # Remove acentos (necessita da biblioteca 'unidecode')
#         new_name_list.append(name)
#     return new_name_list

# def is_similar(a, b):
#     a = parse_name(a)
#     b = parse_name(b)
#     if len(a) != len(b): # Se o número de palavras for diferente, retorna falso
#         return False
#     for x, y in zip(a, b):
#         if (len(x) == 1) or (len(y) == 1): # Se uma das palavras possuir apenas uma letra...
#             if x[0] != y[0]: #...compara apenas a primeira letra
#                 return False
#         else: # Caso contrário...
#             if x != y: #...compara a palavra toda
#                 return False
#     return True # Se todas as palavras forem iguais, retorna verdadeiro

## Comparando o retorno e gerando uma pontuação na saída

In [32]:
df['saida'] = 0

for i in range(len(df['curso esperado'])):
    if df['curso esperado'][i] == df['curso retornado'][i]:
        df['saida'][i] = 1

df

,entrada (português),índice,curso esperado,curso retornado,saida
0,Quais as modalidades de cursos de graduacao de...,0,Art. 1 Os cursos de graduação da Universidade ...,Nas disciplinas cuja avaliação final é realiza...,0
1,Quem estabelece as diretrizes curriculares?,1,Os cursos de graduação poderão oferecer habili...,Será permitido ao aluno provável concluinte no...,0
2,Como sao organizadas as atividades academicas ...,2,Os cursos de graduação terão suas atividades a...,As vagas remanescentes em disciplinas após o e...,0
3,O que visam os cursos de graduacao,3,Os cursos de graduação visam à obtenção de qua...,Nas disciplinas cuja avaliação final é realiza...,0
4,Quem regulamenta a estrutura curricular de cad...,4,A estrutura curricular de cada curso de gradua...,A matrícula em disciplinas far-se-á dentre um ...,0
...,...,...,...,...,...
192,Sou do exterior posso transferir pra ufcg,11,Será permitida a transferência para cursos da ...,"A matrícula em disciplinas será realizada, por...",0
193,Posso transferir pra ufcg,11,Será permitida a transferência para cursos da ...,A matrícula institucional poderá ser concedida...,0
194,Quais modalidades de transferencia,11,Será permitida a transferência para cursos da ...,A abreviação da duração do curso de graduação ...,0
195,O processo seletivo pra entrar na ufcg tem num...,12,A admissão de alunos aos cursos de graduação d...,"A matrícula em disciplinas será realizada, por...",0


In [27]:
df.head(60)

,entrada (português),índice,curso esperado,curso retornado,saida
0,Quais as modalidades de cursos de graduacao de...,0,Art. 1 Os cursos de graduação da Universidade ...,Nas disciplinas cuja avaliação final é realiza...,0
1,Quem estabelece as diretrizes curriculares?,1,Os cursos de graduação poderão oferecer habili...,"A matrícula em disciplinas será realizada, por...",0
2,Como sao organizadas as atividades academicas ...,2,Os cursos de graduação terão suas atividades a...,Aos alunos matriculados na UFCG é permitida ma...,0
3,O que visam os cursos de graduacao,3,Os cursos de graduação visam à obtenção de qua...,Nas disciplinas cuja avaliação final é realiza...,0
4,Quem regulamenta a estrutura curricular de cad...,4,A estrutura curricular de cada curso de gradua...,"A matrícula em disciplinas será realizada, por...",0
5,Quais tipos de cursos podem ser criados,5,"A proposta de criação de cursos, atendidos os ...","A matrícula em disciplinas será realizada, por...",0
6,Projetos de criacao de cursos,6,Os projetos de criação de curso deverão consta...,A solicitação da inclusão em regime de exercíc...,0
7,Extinção ou desativamento dos cursos mantidos ...,7,Os cursos mantidos pela UFCG poderão ser extin...,"A matrícula em disciplinas será realizada, por...",0
8,Como posso ingressar na UFCG?,8,O ingresso nos cursos de graduação da UFCG far...,"A matrícula em disciplinas será realizada, por...",0
9,Os processos seletivos para admissao nos curso...,9,Os processos seletivos para admissão nos curso...,Nas disciplinas cuja avaliação final é realiza...,0


## Pontuação total da métrica

In [33]:
score = sum(df['saida'])

score

0